In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
sys.path.append(os.getcwd() + '/core')
sys.path.append(os.getcwd() + '/core/train_GLOW') 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import json
from datetime import datetime

import core.config as config
from core.model_loader import load_pretrained_VAE, load_pretrained_GLOW
from core.data_loader import TRAIN_loader, TEST_loader
from core.custom_loss import KL_div, VAE_loss
from core.fisher_utils_VAE import Calculate_fisher_VAE, Calculate_score_VAE
from core.fisher_utils_VAE import AUTO_VAE_CIFAR, AUTO_VAE_FMNIST
from core.fisher_utils_GLOW import Calculate_fisher_GLOW, Calculate_score_GLOW
from core.fisher_utils_GLOW import AUTO_GLOW_CIFAR, AUTO_GLOW_FMNIST
from core.visualize import plot_hist, AUROC

# Define Global Variables & Initialize

In [2]:
# FISHER_invs : Fisher inverse matrices from calculating Fisher score (w.r.t. train-dist)
FISHERs = {'VAE': {'cifar10': 0, 'fmnist': 0},
           'GLOW': {'cifar10': 0, 'fmnist': 0}}
         
# NORMalize_FACtors : Normalizing Factor (w.r.t. train-dist)
NORM_FACs = {'VAE': {'cifar10': 0, 'fmnist': 0},
             'GLOW': {'cifar10': 0, 'fmnist': 0}}
             
# SCOREs : (Scalars) Scores from calculating Fisher score (w.r.t. target-dist)
SCOREs = {'VAE': {'cifar10': {}, 'fmnist': {}},
          'GLOW': {'cifar10': {}, 'fmnist': {}}}

# VAE-CIFAR10

In [3]:
train_dist = 'cifar10'
opt = config.VAE_cifar10
ngf, nz, augment = 64, 100, None
netE, netG = load_pretrained_VAE(option=train_dist, ngf=ngf, nz=nz, augment=augment)

#params = ['mu'] # for this choice, you must change the architecture of VAE (go core/train_VAE/DCGAN_VAE_pixel.py line 42)
params = [netE.conv1.weight, netG.main[0].weight, netG.main[-1].weight]
params_name = ['Econv1', 'Gmain0', 'Gmain-1'] # to assign json filename properly
assert len(params) == len(params_name), 'If you modified params, please modify params_name, too!'
max_iter_list = [[300, 100], [500, 200], [1000, 500], [3000, 1000], [10000, 3000], [30000, 5000]]

start = datetime.today()
for max_iter in max_iter_list:
    print(f'Start! {max_iter}')
    a, b, _scores = AUTO_VAE_CIFAR(netE, netG, params, max_iter=max_iter, loss_type='ELBO')

    for i in range(len(params)):
        
        FISHERs['VAE'][train_dist] = a[params[i]]
        NORM_FACs['VAE'][train_dist] = b[params[i]]
        for ood in opt.ood_list:
            SCOREs['VAE'][train_dist][ood] = _scores[ood][params[i]]

        # AUROC curve
        auroc = {}
        for ood in opt.ood_list:
            args = [
                SCOREs['VAE'][train_dist][train_dist],
                SCOREs['VAE'][train_dist][ood],
            ]
            labels = [train_dist, ood]
            auroc[ood] = AUROC(*args, labels=labels, verbose=False)
        print(pd.Series(auroc))

        filename = f'{params_name[i]}_num_sample_{max_iter[0]}_{max_iter[1]}_ngf_{ngf}_nz_{nz}_augment_{augment}'
        with open(f'./results/VAE_{train_dist}/{filename}.json', 'w') as f:
            json.dump(auroc, f)
            
    now = datetime.today()
    print(f'Elapsed time : {now - start}')
    torch.cuda.empty_cache()
    
    
    
path = f'./results/VAE_{train_dist}/'
df = pd.DataFrame()
for file in os.listdir(path):
    if file[-4:] != 'json':
        continue
    temp = file.split('_num_sample_')
    col = (temp[0], f'{temp[1].split("_")[0]}, {temp[1].split("_")[1]}')
    with open(path+file, 'r') as f:
        data = json.load(f)
        df[col] = pd.Series(data)
temp = [f'{elt[0]}, {elt[1]}' for elt in max_iter_list]
df.columns = pd.MultiIndex.from_product([params_name, temp])
df.to_csv(f'{path}result_table.csv')

display(df)

Start! [300, 100]
Files already downloaded and verified


Calculate Fisher VAE:   1%|▎                                                     | 299/50000 [00:15<44:03, 18.80step/s]


Files already downloaded and verified


Calculate Score VAE:   1%|▌                                                       | 99/10000 [00:03<05:08, 32.05step/s]


Using downloaded and verified file: ../data\test_32x32.mat


...

Files already downloaded and verified


Calculate Score VAE:  50%|██████████████████████████▉                           | 4999/10000 [01:16<01:16, 64.97step/s]


Files already downloaded and verified


Calculate Score VAE:  50%|██████████████████████████▉                           | 4999/10000 [01:15<01:15, 66.41step/s]


{'cifar10': 0.5, 'svhn': 0.8421926800000001, 'celeba': 0.8030542599999999, 'lsun': 0.613054, 'cifar100': 0.54201796, 'mnist': 0.9993709199999999, 'fmnist': 0.9933366399999999, 'kmnist': 0.99908524, 'omniglot': 0.99818548, 'notmnist': 0.9968381199999999, 'trafficsign': 0.53232746, 'noise': 0.12112828, 'constant': 0.98965776}
{'cifar10': 0.5, 'svhn': 0.7468202199999999, 'celeba': 0.79835148, 'lsun': 0.6943673333333332, 'cifar100': 0.53726764, 'mnist': 0.99233168, 'fmnist': 0.9791108800000001, 'kmnist': 0.9960083999999999, 'omniglot': 0.9933364, 'notmnist': 0.98513628, 'trafficsign': 0.36535038, 'noise': 0.006196360000000007, 'constant': 0.9693365599999999}
{'cifar10': 0.5, 'svhn': 0.81900026, 'celeba': 0.62912912, 'lsun': 0.42180966666666664, 'cifar100': 0.49758038, 'mnist': 0.8383122, 'fmnist': 0.78797048, 'kmnist': 0.8244875199999999, 'omniglot': 0.6947161800000001, 'notmnist': 0.92652578, 'trafficsign': 0.64576118, 'noise': 0.12147918, 'constant': 0.99954736}
Elapsed time : 1:00:53.12

# VAE-FMNIST

In [4]:
train_dist = 'fmnist'
opt = config.VAE_fmnist
ngf, nz, augment = 32, 100, None
netE, netG = load_pretrained_VAE(option=train_dist, ngf=ngf, nz=nz, augment=augment)

#params = ['mu'] # for this choice, you must change the architecture of VAE (go core/train_VAE/DCGAN_VAE_pixel.py line 42)
params = [netE.conv1.weight, netG.main[0].weight, netG.main[-1].weight]
params_name = ['Econv1', 'Gmain0', 'Gmain-1'] # to assign json filename properly
assert len(params) == len(params_name), 'If you modified params, please modify params_name, too!'
max_iter_list = [[300, 100], [500, 200], [1000, 500], [3000, 1000], [10000, 3000], [30000, 5000]]

start = datetime.today()
for max_iter in max_iter_list:
    print(f'Start! {max_iter}')
    a, b, _scores = AUTO_VAE_FMNIST(netE, netG, params, max_iter=max_iter, loss_type='ELBO')
    
    for i in range(len(params)):

        FISHERs['VAE'][train_dist] = a[params[i]]
        NORM_FACs['VAE'][train_dist] = b[params[i]]
        for ood in opt.ood_list:
            SCOREs['VAE'][train_dist][ood] = _scores[ood][params[i]]

        # AUROC curve
        auroc = {}
        for ood in opt.ood_list:
            args = [
                SCOREs['VAE'][train_dist][train_dist],
                SCOREs['VAE'][train_dist][ood],
            ]
            labels = [train_dist, ood]
            auroc[ood] = AUROC(*args, labels=labels, verbose=False)
        print(pd.Series(auroc))

        filename = f'{params_name[i]}_num_sample_{max_iter[0]}_{max_iter[1]}_ngf_{ngf}_nz_{nz}_augment_{augment}'
        with open(f'./results/VAE_{train_dist}/{filename}.json', 'w') as f:
            json.dump(auroc, f)
    
    now = datetime.today()
    print(f'Elapsed time : {now - start}')
    torch.cuda.empty_cache()
    
    
    
path = f'./results/VAE_{train_dist}/'
df = pd.DataFrame()
for file in os.listdir(path):
    if file[-4:] != 'json':
        continue
    temp = file.split('_num_sample_')
    col = (temp[0], f'{temp[1].split("_")[0]}, {temp[1].split("_")[1]}')
    with open(path+file, 'r') as f:
        data = json.load(f)
        df[col] = pd.Series(data)
temp = [f'{elt[0]}, {elt[1]}' for elt in max_iter_list]
df.columns = pd.MultiIndex.from_product([params_name, temp])
df.to_csv(f'{path}result_table.csv')

display(df)

Calculate Fisher VAE:   0%|                                                                | 0/60000 [00:00<?, ?step/s]

Start! [300, 100]


Calculate Score VAE:   1%|▌                                                       | 99/10000 [00:01<02:53, 56.92step/s]


Using downloaded and verified file: ../data\test_32x32.mat


Calculate Score VAE:  16%|█████████▌                                                | 99/600 [00:01<00:05, 87.46step/s]


...

Calculate Score VAE:  50%|██████████████████████████▍                          | 4999/10000 [00:45<00:45, 109.40step/s]


Files already downloaded and verified


Calculate Score VAE:  50%|██████████████████████████▍                          | 4999/10000 [00:44<00:44, 112.82step/s]

fmnist      0.500000
svhn        0.999993
celeba      0.999726
lsun        0.999844
cifar10     0.999556
cifar100    0.999465
mnist       0.983381
kmnist      0.993261
omniglot    1.000000
notmnist    0.999776
noise       0.995179
constant    0.997080
dtype: float64
fmnist      0.500000
svhn        0.999980
celeba      0.999862
lsun        0.999832
cifar10     0.999632
cifar100    0.999529
mnist       0.966025
kmnist      0.994394
omniglot    1.000000
notmnist    0.999831
noise       0.977015
constant    0.997009
dtype: float64
fmnist      0.500000
svhn        0.955290
celeba      0.817419
lsun        0.721367
cifar10     0.718826
cifar100    0.724923
mnist       0.542106
kmnist      0.764547
omniglot    1.000000
notmnist    0.976339
noise       0.999499
constant    0.997161
dtype: float64
Elapsed time : 0:27:01.269822


Econv1                                                         \
          300, 100  500, 200 1000, 500 3000, 1000 10000, 3000 30000, 5000   
fmnist    0.500000  0.500000  0.500000   0.500000      0.5000    0.500000   
svhn      0.999949  1.000000  0.999993   0.999922      1.0000    0.999250   
celeba    0.999553  0.999696  0.999726   0.999403      0.9986    0.998025   
lsun      0.999566  0.999872  0.999844   0.999557      0.9991    0.997975   
cifar10   0.999336  0.999616  0.999556   0.999386      0.9987    0.998325   
cifar100  0.999108  0.999384  0.999465   0.999286      0.9983    0.998275   
mnist     0.981063  0.980548  0.983381   0.982518      0.9837    0.988200   
kmnist    0.992745  0.988400  0.993261   0.992844      0.9925    0.990400   
omniglot  1.000000  1.000000  1.000000   1.000000      1.0000    1.000000   
notmnist  0.999726  0.999944  0.999776   0.999700      0.9998    0.997950   
noise     0.992457  0.996068  0.995179   0.995570      1.0000    0.995000   
constant  0.997128  0.994004  0.997080   0.995895      0.9999    0.999275   

            Gmain0                                                         \
          300, 100  500, 200 1000, 500 3000, 1000 10000, 3000 30000, 5000   
fmnist    0.500000  0.500000  0.500000   0.500000      0.5000    0.500000   
svhn      0.953179  0.927172  0.955290   0.946797      0.6223    0.785950   
celeba    0.821770  0.832008  0.817419   0.825745      0.4917    0.626575   
lsun      0.741524  0.794196  0.721367   0.783483      0.4318    0.641800   
cifar10   0.745331  0.774052  0.718826   0.757908      0.4889    0.655725   
cifar100  0.736726  0.778064  0.724923   0.757121      0.4961    0.651175   
mnist     0.672422  0.884112  0.542106   0.791038      0.4943    0.779400   
kmnist    0.856436  0.909604  0.764547   0.893809      0.5071    0.789625   
omniglot  1.000000  1.000000  1.000000   1.000000      1.0000    1.000000   
notmnist  0.981541  0.989720  0.976339   0.984748      0.8810    0.972425   
noise     0.999634  0.973504  0.999499   0.998992      0.2530    0.671950   
constant  0.997023  0.988356  0.997161   0.994776      0.9471    0.975850   

           Gmain-1                                                         
          300, 100  500, 200 1000, 500 3000, 1000 10000, 3000 30000, 5000  
fmnist    0.500000  0.500000  0.500000   0.500000      0.5000    0.500000  
svhn      0.999928  1.000000  0.999980   0.999844      1.0000    0.999925  
celeba    0.999743  0.999888  0.999862   0.999638      1.0000    0.999600  
lsun      0.999576  0.999952  0.999832   0.999447      1.0000    0.999300  
cifar10   0.999369  0.999748  0.999632   0.999384      1.0000    0.999325  
cifar100  0.999192  0.999624  0.999529   0.999356      0.9996    0.999025  
mnist     0.963354  0.959728  0.966025   0.966647      0.9631    0.958025  
kmnist    0.993406  0.990324  0.994394   0.994334      0.9954    0.991075  
omniglot  1.000000  1.000000  1.000000   1.000000      1.0000    1.000000  
notmnist  0.999800  0.999948  0.999831   0.999737      1.0000    0.999800  
noise     0.975404  0.971348  0.977015   0.980620      0.9705    0.977800  
constant  0.996973  0.993828  0.997009   0.995500      1.0000    0.999500

# GLOW-CIFAR10

In [ ]:
train_dist = 'cifar10'
opt = config.GLOW_cifar10
model = load_pretrained_GLOW(option=train_dist)

dicts = [model.flow.layers[-1]]

In [ ]:
a, b, c, d, e, f, g = AUTO_GLOW_CIFAR(model, dicts)
GRADs['GLOW']['cifar10'] = a
NORM_FACs['GLOW']['cifar10'] = b
Gradients['GLOW']['cifar10']['cifar10'] = c
Gradients['GLOW']['cifar10']['svhn'] = d
Gradients['GLOW']['cifar10']['celeba'] = e
Gradients['GLOW']['cifar10']['lsun'] = f
Gradients['GLOW']['cifar10']['noise'] = g

# GLOW-FMNIST